In [1]:
import os
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql import SparkSession

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
conf = SparkConf().setAppName('Spark DL Tabular Pipeline').setMaster('local[6]')
sc = SparkContext(conf=conf)
sql_context = SparkSession(sc)

In [2]:
# Load Data to Spark Dataframe
df = sql_context.read.csv('amazon_reviews.tsv',
                          header=True,
                          sep=r'\t',
                          inferSchema=True)
df.printSchema()



root
 |-- DOC_ID: integer (nullable = true)
 |-- LABEL: string (nullable = true)
 |-- RATING: integer (nullable = true)
 |-- VERIFIED_PURCHASE: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- PRODUCT_TITLE: string (nullable = true)
 |-- REVIEW_TITLE: string (nullable = true)
 |-- REVIEW_TEXT: string (nullable = true)



In [3]:
df.describe().show()
from pyspark.sql.functions import col, lower, regexp_replace, split

# Convert to Lower Case, remove html tags
def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "<.*?>", "")
  # c = regexp_replace(c, "<.*?> ", "")
  return c

clean_text_df = df.select(clean_text(col("REVIEW_TEXT")).alias("REVIEW_TEXT"))

clean_text_df.printSchema()
clean_text_df.show(10)



+-------+-----------------+----------+-----------------+-----------------+----------------+--------------------+--------------------+--------------------+-----------------------------------+
|summary|           DOC_ID|     LABEL|           RATING|VERIFIED_PURCHASE|PRODUCT_CATEGORY|          PRODUCT_ID|       PRODUCT_TITLE|        REVIEW_TITLE|                        REVIEW_TEXT|
+-------+-----------------+----------+-----------------+-----------------+----------------+--------------------+--------------------+--------------------+-----------------------------------+
|  count|            21000|     21000|            21000|            21000|           21000|               21000|               21000|               21000|                              21000|
|   mean|          10500.5|      null|4.127952380952381|             null|            null|1.2152996572572742E9|                 9.0|              1421.0|                               null|
| stddev|6062.322162340104|      null|1.27833

In [4]:
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

tokenizer = Tokenizer(inputCol="REVIEW_TEXT", outputCol="words")

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(clean_text_df)
tokenized.select("REVIEW_TEXT", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)




ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

ngramDataFrame = ngram.transform(tokenized)
ngramDataFrame.select("ngrams").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml import Pipeline
w2v = Word2Vec(vectorSize=300, minCount=0, inputCol="words", outputCol="features")
doc2vec_pipeline = Pipeline(stages=[w2v])
doc2vec_model = doc2vec_pipeline.fit(tokenized)
doc2vecs_df = doc2vec_model.transform(tokenized)
doc2vecs_df.show()

Py4JJavaError: An error occurred while calling o95.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 21, charles-HP-Z420-Workstation.cable.rcn.com, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1786)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1189)
	at java.io.ObjectOutputStream.writeObject(ObjectOutput

In [ ]:
tokenized.head()